<a href="https://colab.research.google.com/github/Harshal-Kharkar/Amazon/blob/main/track_and_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


In [ ]:

from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [ ]:
%cd {HOME}
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt


/content
--2025-01-09 09:35:29--  https://docs.google.com/uc?export=download&confirm=&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-
Resolving docs.google.com (docs.google.com)... 142.251.179.138, 142.251.179.102, 142.251.179.113, ...
Connecting to docs.google.com (docs.google.com)|142.251.179.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-&export=download [following]
--2025-01-09 09:35:29--  https://drive.usercontent.google.com/download?id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.179.132, 2607:f8b0:4004:c1f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.179.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35345757 (34M) [video/mp4]
Saving to: ‘vehicle-counting.mp4’

vehicle-counting.mp 100%[===================>]  33.71M

In [ ]:
SOURCE_VIDEO_PATH = f"{HOME}/vehicle-counting.mp4"

In [ ]:
!pip install ultralytics

In [ ]:
from IPython.display import display
import ultralytics

In [ ]:
ultralytics.checks()

Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.4/107.7 GB disk)


In [ ]:
!pip install supervision

In [ ]:
import supervision as sv

In [ ]:
print(sv.__version__)

0.25.1


In [ ]:
model = ultralytics.YOLO('yolov8n.pt')
model.fuse()

YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


In [ ]:
%cd {HOME}
!yolo detect predict model=yolov8n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg'

/content
Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

Found https://media.roboflow.com/notebooks/examples/dog.jpeg locally at dog.jpeg
image 1/1 /content/dog.jpeg: 640x384 1 person, 1 car, 1 dog, 334.8ms
Speed: 4.7ms preprocess, 334.8ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs/detect/predict5
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
%cd {HOME}
!yolo detect predict model=yolov8n.pt conf=0.25 source='/content/vehicle-counting.mp4'

/content
Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 5, in <module>
    from ultralytics.cfg import entrypoint
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/__init__.py", line 11, in <module>
    from ultralytics.models import NAS, RTDETR, SAM, YOLO, FastSAM, YOLOWorld
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/models/__init__.py", line 3, in <module>
    from .fastsam import FastSAM
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/models/fastsam/__init__.py", line 3, in <module>
    from .model import FastSAM
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/models/fastsam/model.py", line 5, in <module>
    from ultralytics.engine.model import Model
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 8, in <module>
    import torch
  File "/usr/local/lib/python3.10/dist-packages/torch/__init__.py", line 2486, in <module>
    from torch import _meta_registrations
  File "/usr/lo

In [ ]:
# import supervision as sv
# # from supervision.video.source import get_video_frames_generator
# from tqdm.notebook import tqdm

# # Initialize annotators
# box_annotator = sv.BoxAnnotator(color=sv.ColorPalette.from_hex(['#ff0000', '#00ff00']), thickness=4)
# label_annotator = sv.LabelAnnotator(
#     text_color=sv.Color.BLACK,  # Corrected: sv.Color.BLACK
#     text_scale=2,  # Adjusted for better visibility
#     text_thickness=6,  # Increased text thickness
#     text_padding=5,  # Added padding
#       # Corrected: Specify the background color
# )

# # Video source and sink

# VIDEO_OUTPUT_PATH = "output_video.mp4"  # Path to save the output video

# # Generate frames from the source video
# gen = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
# gen = iter(gen)

# # Get video information
# video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# # Model setup (Ensure you have a model initialized; replace with your actual model code)
# # Example: Using YOLO or another object detection model
# # model = YourModel()  # Replace with the actual model

# with sv.VideoSink(VIDEO_OUTPUT_PATH, video_info) as sink:
#     for frame in tqdm(gen, total=video_info.total_frames):  # Iterate through frames
#         # Perform inference on the frame (replace with your model's specific prediction code)
#         results = model(frame)[0]

#         # Extract detections
#         detections = sv.Detections(
#             xyxy=results.boxes.xyxy.cpu().numpy(),
#             confidence=results.boxes.conf.cpu().numpy(),
#             class_id=results.boxes.cls.cpu().numpy().astype(int)
#         )

#         # Step 1: Annotate the frame with bounding boxes
#         frame = box_annotator.annotate(scene=frame, detections=detections)

#         # Step 2: Annotate the frame with labels
#         labels = [
#             f"{model.names[class_id]} {confidence:0.2f}"
#             for confidence, class_id in zip(detections.confidence, detections.class_id)
#         ]
#         frame = label_annotator.annotate(scene=frame, detections=detections, labels=labels)

#         # Write the annotated frame to the video sink
#         sink.write_frame(frame)


In [ ]:
from typing import List

import numpy as np


# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis].astype(np.float64)  # Ensure dtype is correct
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

In [ ]:

# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
# class_ids of interest - car, motorcycle, bus and truck
CLASS_ID = [2, 3, 5, 7]

In [ ]:
import numpy as np
np.float = float  # Override np.float


In [ ]:
import torch

In [ ]:
torch.mps.device_count()

0

In [ ]:
len(tracker_id_dict)

27

In [ ]:

from collections import defaultdict


# Create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())


VIDEO_OUTPUT_PATH = "output_video.mp4"
# Get video info
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Create BoxAnnotator
box_annotator = sv.BoxAnnotator(color=sv.ColorPalette.from_hex(['#ff0000', '#00ff00']))

# Create a LineZone for counting objects that cross a line
start, end = sv.Point(x=0, y=1080), sv.Point(x=3840, y=1080)
line_zone = sv.LineZone(start=start, end=end)

# Default line color (white)
line_color = (255, 255, 255)

# Counters for In and Out events
in_count = 0
out_count = 0

# Open target video file
with sv.VideoSink(VIDEO_OUTPUT_PATH, video_info) as sink:
    # Generate frames from the source video
    gen = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
    gen = iter(gen)

    # Initialize a dictionary to store class counts
    class_counts = defaultdict(int)

    # Initialize a tracker for sequential IDs
    tracker_id_counter = 1
    tracker_id_dict = {}

    # Loop over video frames
    for frame in gen:
        # Model prediction on the frame
        results = model(frame, verbose=False)
        detections = sv.Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int)
        )

        # Filtering out detections with unwanted classes (optional)
        mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
        detections.xyxy = detections.xyxy[mask]
        detections.confidence = detections.confidence[mask]
        detections.class_id = detections.class_id[mask]

        if hasattr(detections, 'tracker_id') and detections.tracker_id is not None:
            detections.tracker_id = detections.tracker_id[mask]

        # Tracking detections using BYTETracker
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )
        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)

        # Assign sequential tracker IDs if needed
        for i, tracker_id in enumerate(detections.tracker_id):
            if tracker_id not in tracker_id_dict:
                tracker_id_dict[tracker_id] = tracker_id_counter
                tracker_id_counter += 1  # Increment the counter for the next unique ID
            detections.tracker_id[i] = tracker_id_dict[tracker_id]

        # Filter out detections without trackers
        mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
        detections.xyxy = detections.xyxy[mask]
        detections.confidence = detections.confidence[mask]
        detections.class_id = detections.class_id[mask]
        if hasattr(detections, 'tracker_id'):
            detections.tracker_id = detections.tracker_id[mask]

        # Count the occurrences of each class in the frame
        for class_id in detections.class_id:
            class_counts[class_id] += 1

        # Format custom labels with sequential tracker IDs
        labels = [
            f"ID: {tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
            for confidence, class_id, tracker_id in zip(detections.confidence, detections.class_id, detections.tracker_id)
        ]

        # Annotate the frame with bounding boxes and labels
        frame = box_annotator.annotate(scene=frame, detections=detections)
        frame = label_annotator.annotate(scene=frame, detections=detections, labels=labels)

        # Trigger the line crossing event
        crossed_in, crossed_out = line_zone.trigger(detections)

        # Update counts if objects cross the line
        if True in crossed_in:
            in_count += 1
        if True in crossed_out:
            out_count += 1

        # Change line color if a crossing event occurs
        if True in crossed_in or True in crossed_out:
            line_color = (0, 255, 0)  # Change to green
        else:
            line_color = (255, 255, 255)  # Default to white

        # Draw the line on the frame
        cv2.line(frame, (start.x, start.y), (end.x, end.y), line_color, thickness=3)

        # Display In and Out counts on the frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, f"In: {in_count}", (50, 50), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f"Out: {out_count}", (50, 100), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

        # Write the annotated frame to the output video
        sink.write_frame(frame)
